In [1]:
import math
import torch
from torch_geometric.data import InMemoryDataset, Data, Batch
from torch.utils.data import DataLoader, Subset, random_split
from sklearn.calibration import LabelEncoder
import json 

c:\Users\Abderrahmane\ProjectsC\ReconnaissanceAlgebrique\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [461]:
from torch import nn
from torch.nn import Linear, ReLU, LeakyReLU, init
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, GATv2Conv, TransformerConv, LayerNorm
import numpy as np
import torch.optim.lr_scheduler as lr_scheduler


In [3]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [556]:
DATASET_FILEPATH = "math_datagen_triplet_40k.json"
BATCH_SIZE = 256
TRAIN_RATIO = 0.9
VAL_RATIO = 0.1
DISTANCE_FUNC = F.pairwise_distance
margin = 0.8
rep = 16

In [234]:
TYPES = ["FUNC", "OPERATION", "POW", "LITERAL", "VARIABLE", "CONSTANT_LITERAL"]
OPERATIONS = ["ADD", "MUL", "FUNC", "POW"]
FUNCTIONS = ["SIN", "COS", "TAN", "EXP", "LOG", "SINH", "COSH", "TANH"]
VARIABLE_ALPHABET = [chr(x) for x in range(ord('a'), ord('z')+1) if chr(x) not in ["e", "i"]]
CLASSIC_CONSTANTS = ["PI", "I", "g","e", "zoo", "E"]

In [235]:

def dict_to_geometric_representation(in_graph_dict: dict, encoder) -> Data:
    node_list = []
    edge_mappings = []
    def traverse_graph(graph = in_graph_dict):
        nonlocal node_list
        nonlocal edge_mappings
        curr_node_index = len(node_list)
        encoded_data = encoder({"type": graph["type"], "subtype": graph["subtype"], "value": graph["value"]})
        # print("Encoded data: ", encoded_data)
        node_list.append(encoded_data)
        if "children" in graph.keys():
            for child in graph["children"]:
                edge_mappings.append((curr_node_index, traverse_graph(child)) ) #I'm retarded.
        return curr_node_index
    traverse_graph()
    nodes = torch.tensor(node_list,dtype=torch.float32)
    if not torch.all(torch.isfinite(nodes)):
        print("NODES CONTAIN INFINITE VALUE ---> \n")
        print("X = ", nodes)
    nodes[nodes == float("inf")] = 0
    nodes[nodes == -float("inf")] = 0
    if not torch.all(torch.isfinite(nodes)):
        print("MODIFIED NODES CONTAIN INFINITE VALUE ---> \n")
        print("X = ", nodes)
    edges = torch.tensor([[x[0] for x in edge_mappings], [x[1] for x in edge_mappings]], dtype=torch.long) # Probably slow and mentally degenerated
    geom_data = Data(x=nodes, edge_index=edges)
    return geom_data

#### Encoder

In [236]:
label_encoder = LabelEncoder()
label_encoder.fit(OPERATIONS+FUNCTIONS+CLASSIC_CONSTANTS+VARIABLE_ALPHABET+TYPES)
# --- misc
nan_count = 0 
sample_count = 0

In [237]:
def sanitize_value(value):
    if value in ["-Infinity","Infinity", float("inf"), float("-inf"), "inf", "-inf"]:
        return None
    return value

In [238]:
def node_attr_encoder(attr):
    global nan_count
    global sample_count
    
    def safe_label_encode(value):
        return label_encoder.transform([value])[0] if value else -1

    type_encoding = [safe_label_encode(attr.get("type"))] * rep
    subtype_encoding = [safe_label_encode(attr.get("subtype"))] * rep
    
    value_encoding_vec = [-1] * rep  # Default to all -1 if no valid value
    
    if sanitize_value(attr.get("value")) is not None:
        if isinstance(attr["value"], str):
            value_encoding = safe_label_encode(attr["value"])
            value_encoding_vec = [(0 if i % 2 == 0 else value_encoding) for i in range(rep)]
        else:
            try:
                val = float(attr["value"])
                if math.isfinite(val):
                    expon = math.log10(abs(val))
                    val = val // 10**int(expon)
                    value_encoding_vec = [(expon if i % 2 == 1 else val) for i in range(rep)]
                else:
                    raise ValueError("Non-finite value")
            except (ValueError, TypeError):
                nan_count += 1
                value_encoding_vec = [(-1 if i % 2 == 1 else 1) for i in range(rep)]
            finally:
                sample_count += 1

    result = type_encoding + subtype_encoding + value_encoding_vec
    return result

#### Loading in the dataset

In [239]:
triplet_dataset = []

In [240]:
# print('"value": Infinity,'.find("Infinity"))

In [241]:
with open(DATASET_FILEPATH) as file_handle:
    object_data = json.load(file_handle)
    for triplet in object_data:
        ## TODO : check if contains infinity (triplet to str and then search for Infinity)
        # print(triplet)
        # print(json.dumps(triplet).find("inf"))
        
        if json.dumps(triplet).find("inf") == -1:
            expr_true = triplet["expr_true"]
            expr_false = triplet["expr_false"]
            expr_anchor = triplet["expr_anchor"]
            geom_anchor = dict_to_geometric_representation(expr_anchor,node_attr_encoder )
            geom_true = dict_to_geometric_representation(expr_true,node_attr_encoder )
            geom_false = dict_to_geometric_representation(expr_false,node_attr_encoder )
            triplet_dataset.append((geom_anchor, geom_true, geom_false))
        
        

In [242]:
# with open("test","w") as f:
#     f.write(str(triplet_dataset))

#### Neural network training

In [294]:
num_features = triplet_dataset[0][0].num_features

In [504]:
class FormulaNet(nn.Module):
    def __init__(self, hidden_channels: int, embedding_space: int):
        super(FormulaNet, self).__init__()
        self.dense_1 = Linear(num_features, num_features*2) 
        self.dense_2 = Linear(num_features*2, hidden_channels) 
        
        self.relu_1 = LeakyReLU(0.2)
        self.gconv_1 = GCNConv(num_features*2, hidden_channels)
        self.gconv_2 = GCNConv(hidden_channels, hidden_channels)
        self.trans1 = TransformerConv(hidden_channels, hidden_channels, heads=1)
        self.trans2 = TransformerConv(hidden_channels, hidden_channels, heads=1)
        self.layer_norm = LayerNorm(in_channels=hidden_channels)
        
        self.dense_3 = Linear(hidden_channels, embedding_space)
        gain = torch.nn.init.calculate_gain("leaky_relu", 0.2)
        # Initialize dense_1 weights
        init.xavier_uniform_(self.dense_1.weight, gain=gain)
        init.xavier_uniform_(self.dense_2.weight, gain=gain)
        # # Initialize gconv_1 weights
        init.xavier_uniform_(self.gconv_1.lin.weight.data.T, gain=gain)
        # # Initialize gconv_2 weights
        init.xavier_uniform_(self.gconv_2.lin.weight.data.T, gain=gain)
        #Initialize dense_3 weights
        init.xavier_uniform_(self.dense_3.weight, gain=gain)
    
    def forward(self, x, edge_index, batch):
        if not torch.all(torch.isfinite(x)):
            print("INPUT CONTAINS INFINITE VALUE")
            #print("x = ", x)
            


        nan_mask = torch.isnan(x)
        if nan_mask.any():
            print("Some values of the input are nan: ", x[nan_mask])
            # Replace NaNs with 0s
            x[nan_mask] = -11
            
        
        xd = self.dense_1(x)
        xd = self.relu_1(xd)
        
        xd = self.dense_2(xd)
        xd = self.relu_1(xd)
        xc = xd
        # xc = self.gconv_1(xd, edge_index)
        # xc = self.relu_1(xc)
        # xc = self.gconv_2(xc, edge_index)
        # xc = self.relu_1(xc)
        xt = self.trans1(xc, edge_index)
        xt = self.layer_norm(xt)
        xt = self.relu_1(xt) + xc
        
        xt2 = self.trans2(xt, edge_index)
        xt2 = self.layer_norm(xt2)
        xt2 = self.relu_1(xt2) + xt
        
        # xc = self.gconv_3(xc, edge_index)
        # xc = self.relu_1(xc)
        xp = global_mean_pool(xt, batch)
        xp = F.dropout(xp, p=0.4,training=self.training)
        xp = self.dense_3(xp)
        xp = self.relu_1(xp)
        # nan_mask = torch.isnan(x)
        # if nan_mask.any():
        #     print("Some values of the input are nan: ", x[nan_mask])
            
        return xp
    

In [505]:
class TripletFormulaNet(nn.Module):
    def __init__(self, hidden_channels, embedding_space):
        super(TripletFormulaNet, self).__init__()
        self.formulanet = FormulaNet(hidden_channels, embedding_space)

    def forward(self, anchor, positive, negative):
        # Compute the embeddings for the anchor, positive, and negative inputs
        embed_anchor = self.formulanet(anchor.x, anchor.edge_index, anchor.batch)
        embed_positive = self.formulanet(positive.x, positive.edge_index, positive.batch)
        embed_negative = self.formulanet(negative.x, negative.edge_index, negative.batch)
        
        return embed_anchor, embed_positive, embed_negative

#### Training

In [506]:
# ## CHECK DATASET:
# with open("test", "w") as f:
#     for a, b,c in triplet_dataset:
#         f.write(str(a.x) + "\n" + str(b.x) + "\n" + str(c.x) + "\n")

In [557]:
def triplet_loss(anchor, positive, negative, margin=1.0):
    """
    Triplet loss function.
    
    Args:
        anchor (torch.Tensor): Embedding of the anchor data point.
        positive (torch.Tensor): Embedding of the positive data point (same class as anchor).
        negative (torch.Tensor): Embedding of the negative data point (different class from anchor).
        margin (float, optional): Margin value for triplet loss. Defaults to 1.0.
        
    Returns:
        torch.Tensor: Triplet loss value.
    """
    
    # Calculate the Euclidean distances between the embeddings
    distance_positive = DISTANCE_FUNC(anchor, positive)
    distance_negative = DISTANCE_FUNC(anchor, negative)
    
    # Calculate the triplet loss
    losses = F.relu(distance_positive - distance_negative + margin)
    
    return losses.mean()

In [508]:
total_len = len(triplet_dataset)
train_len = int(TRAIN_RATIO * total_len)
val_len = int(VAL_RATIO * total_len)
test_len = total_len - train_len - val_len

train_dataset, val_dataset, test_dataset = random_split(triplet_dataset, [train_len, val_len, test_len])

In [509]:
def collate(data_list):
    batchA = Batch.from_data_list([data[0] for data in data_list])
    batchB = Batch.from_data_list([data[1] for data in data_list])
    batchC = Batch.from_data_list([data[2] for data in data_list])
    return batchA, batchB, batchC

In [510]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
validation_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle= False, collate_fn=collate)

In [511]:
device = torch.device("cpu") #torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [512]:
model = TripletFormulaNet(32,32).to(device)

In [513]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.08)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.01, total_iters=20)

In [514]:
from tqdm import tqdm

In [515]:
training_loss = []  # Store training loss per epoch
validation_loss = []  # Store validation loss per epoch

In [558]:
def train(epoch):
    model.train()
    epoch_loss = 0
    for batch_anchor, batch_positive, batch_negative in tqdm(train_loader, desc=f'Epoch {epoch}'):

        batch_anchor, batch_positive, batch_negative = batch_anchor.to(device), batch_positive.to(device), batch_negative.to(device)
        optimizer.zero_grad()
        embed_anchor, embed_positive, embed_negative = model(batch_anchor, batch_positive, batch_negative)
        loss = triplet_loss(embed_anchor, embed_positive, embed_negative, margin=margin)
        loss.backward()
        writer.add_scalar("Loss/train", loss, epoch)
        
        # Log gradients
        for name, param in model.formulanet.named_parameters():
            if param.grad is not None and param.grad.numel() > 0:
                writer.add_histogram(f'{name}.grad', param.grad, epoch)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()
    scheduler.step(epoch)
    training_loss.append(epoch_loss / len(train_loader))  # Store epoch average loss
    print(f'Epoch {epoch}, Loss: {epoch_loss / len(train_loader)}')

In [665]:
def validate(model, validation_loader, device):
    model.eval()
    triplet_loss_tot = 0
    correct_triplets = 0
    total_triplets = 0
    total_dist_pos = 0
    total_dist_neg = 0
    

    with torch.no_grad():
        for batch_anchor, batch_positive, batch_negative in validation_loader:
            batch_anchor, batch_positive, batch_negative = batch_anchor.to(device), batch_positive.to(device), batch_negative.to(device)
            embed_anchor, embed_positive, embed_negative = model(batch_anchor, batch_positive, batch_negative)

            # Compute triplet loss
            loss = triplet_loss(embed_anchor, embed_positive, embed_negative, margin=margin)
            triplet_loss_tot += loss.item() * batch_anchor.size(0)

            # Compute triplet accuracy
            distance_positive = DISTANCE_FUNC(embed_anchor, embed_positive)
            total_dist_pos += distance_positive.sum().item()
            distance_negative = DISTANCE_FUNC(embed_anchor, embed_negative)
            total_dist_neg += distance_negative.sum().item()
            correct_triplets += (distance_positive < distance_negative).sum().item() ## WARNING: FLIP THIS FOR PAIRWISE DISTANCE
            total_triplets += distance_positive.size(0)

    triplet_loss_tot /= len(validation_loader.dataset)
    triplet_accuracy = correct_triplets / total_triplets

    print(f'Validation Triplet Loss: {triplet_loss_tot}, Total Positive: {total_dist_pos}, Total Negative: {total_dist_neg}, Triplet Accuracy: {triplet_accuracy}')

    return triplet_loss_tot, triplet_accuracy

In [666]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [667]:
num_epochs = 180

for epoch in range(num_epochs):
    train(epoch)
    validate(model, validation_loader, device)

Epoch 0: 100%|██████████| 242/242 [00:21<00:00, 11.51it/s]
C:\Users\Abderrahmane\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 0, Loss: 30.063697422092613
Validation Triplet Loss: 0.0, Total Positive: 7992883.0, Total Negative: 344426187.0, Triplet Accuracy: 1.0


Epoch 1: 100%|██████████| 242/242 [00:28<00:00,  8.59it/s]


Epoch 1, Loss: 481.9965731122277
Validation Triplet Loss: 230.98377247522043, Total Positive: 27894855.46875, Total Negative: 276834152.0, Triplet Accuracy: 0.9988372093023256


Epoch 2: 100%|██████████| 242/242 [00:25<00:00,  9.56it/s]


...

Epoch 109, Loss: 0.06863889024277364
Validation Triplet Loss: 0.0, Total Positive: 7765618.546875, Total Negative: 383455684.0, Triplet Accuracy: 1.0


Epoch 110: 100%|██████████| 242/242 [00:23<00:00, 10.42it/s]


Epoch 110, Loss: 0.002158798708403406
Validation Triplet Loss: 0.0, Total Positive: 7787292.21875, Total Negative: 382881189.0, Triplet Accuracy: 1.0


Epoch 111:  59%|█████▊    | 142/242 [00:16<00:11,  8.58it/s]


KeyboardInterrupt: 

In [668]:
torch.save(model.state_dict(), "./full_model_state_v6")

In [669]:
torch.save(model, "./full_model_v6")

## Testing

In [775]:
from generate_dataset import node_to_dict, sympy_to_node
import sympy as sp
from latex2sympy2 import latex2sympy

In [776]:
def latex_to_graph(tex):
    return node_to_dict(sympy_to_node(latex2sympy(tex)))

In [851]:
expr_l = latex_to_graph(r"x + y")
expr_r = latex_to_graph(r"x + y ")

In [852]:
expr_l

{'type': 'OPERATION',
 'value': None,
 'subtype': 'ADD',
 'children': [{'type': 'VARIABLE',
   'value': 'x',
   'subtype': None,
   'children': []},
  {'type': 'VARIABLE', 'value': 'y', 'subtype': None, 'children': []}]}

In [853]:
expr_l

{'type': 'OPERATION',
 'value': None,
 'subtype': 'ADD',
 'children': [{'type': 'VARIABLE',
   'value': 'x',
   'subtype': None,
   'children': []},
  {'type': 'VARIABLE', 'value': 'y', 'subtype': None, 'children': []}]}

In [854]:
expr_l_obj = dict_to_geometric_representation(expr_l, node_attr_encoder)
expr_r_obj = dict_to_geometric_representation(expr_r, node_attr_encoder)

In [858]:
expr_l_obj.x

tensor([[11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1.],
        [18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18.,
         18., 18., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1.,  0., 41.,  0., 41.,  0., 41.,  0., 41.,  0., 41.,
          0., 41.,  0., 41.,  0., 41.],
        [18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18.,
         18., 18., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1.,  0., 42.,  0., 42.,  0., 42.,  0., 42.,  0., 42.,
          0., 42.,  0., 42.,  0., 42.]])

In [856]:
emb_l, emb_r, emb_r_c = model(expr_l_obj, expr_r_obj, expr_r_obj)

In [850]:
print(F.pairwise_distance(emb_l, emb_r))

tensor([96952.6172], grad_fn=<NormBackward1>)


In [767]:
expr_left = {
      "type": "POW",
      "value": None,
      "subtype": None,
      "children": [
        {
          "type": "POW",
          "value": None,
          "subtype": None,
          "children": [
            {
              "type": "LITERAL",
              "value": -3.5,
              "subtype": None,
              "children": []
            },
            {
              "type": "LITERAL",
              "value": 3.0,
              "subtype": None,
              "children": []
            }
          ]
        },
        {
          "type": "LITERAL",
          "value": 4.5,
          "subtype": None,
          "children": []
        }
      ]
    }
expr_right = {
      "type": "POW",
      "value": None,
      "subtype": None,
      "children": [
        {
          "type": "OPERATION",
          "value": None,
          "subtype": "ADD",
          "children": [
            {
              "type": "LITERAL",
              "value": -5.5,
              "subtype": None,
              "children": []
            },
            {
              "type": "LITERAL",
              "value": -10.0,
              "subtype": None,
              "children": []
            }
          ]
        },
        {
          "type": "LITERAL",
          "value": 4.0,
          "subtype": None,
          "children": []
        }
      ]
    }

expr_right_constant_error ={
      "type": "POW",
      "value": None,
      "subtype": None,
      "children": [
        {
          "type": "OPERATION",
          "value": None,
          "subtype": "ADD",
          "children": [
            {
              "type": "LITERAL",
              "value": -5.5,
              "subtype": None,
              "children": []
            },
            {
              "type": "LITERAL",
              "value": -10.0,
              "subtype": None,
              "children": []
            }
          ]
        },
        {
          "type": "LITERAL",
          "value": 4.0,
          "subtype": None,
          "children": []
        }
      ]
    }

In [302]:
expr_l_obj = dict_to_geometric_representation(expr_left, node_attr_encoder)
expr_r_obj = dict_to_geometric_representation(expr_right, node_attr_encoder)
expr_r_c_obj = dict_to_geometric_representation(expr_right_constant_error, node_attr_encoder)

In [303]:
emb_l, emb_r, emb_r_c = model(expr_l_obj, expr_r_obj, expr_r_c_obj)

In [304]:
print(DISTANCE_FUNC(emb_l, emb_r))
print(DISTANCE_FUNC(emb_l, emb_r_c))
print(DISTANCE_FUNC(emb_r, emb_r_c))
print(DISTANCE_FUNC(emb_r, emb_r))



tensor([1.0000], grad_fn=<SumBackward1>)
tensor([1.0000], grad_fn=<SumBackward1>)
tensor([1.0000], grad_fn=<SumBackward1>)
tensor([1.0000], grad_fn=<SumBackward1>)


In [774]:
model.info()

AttributeError: 'TripletFormulaNet' object has no attribute 'info'

In [ ]:
    {
        "expr_anchor": {"type": "OPERATION", "value": null, "subtype": "ADD", "children": [{"type": "FUNC", "value": null, "subtype": "COS", "children": [{"type": "POW", "value": null, "subtype": null, "children": [{"type": "VARIABLE", "value": "x", "subtype": null, "children": []}, {"type": "LITERAL", "value": 2.0, "subtype": null, "children": []}]}]}, {"type": "FUNC", "value": null, "subtype": "SIN", "children": [{"type": "POW", "value": null, "subtype": null, "children": [{"type": "VARIABLE", "value": "x", "subtype": null, "children": []}, {"type": "LITERAL", "value": 2.0, "subtype": null, "children": []}]}]}]},

        "expr_true": {"type": "LITERAL", "value": 1.0, "subtype": null, "children": []},
        "expr_false": {"type": "FUNC", "value": null, "subtype": "COS", "children": [{"type": "VARIABLE", "value": "x", "subtype": null, "children": []}]}

    }